In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf 
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
df=pd.read_csv("labels.csv")

In [3]:
df.head()

,id,hemorrhage
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [4]:
files=glob.glob("data/raw/hct/imgs/*.png")
SIZE = 256

In [5]:
label=[]
img_array=[]
for i in files:
  sub_heading=i.split('/')
  sub_sub_heading=sub_heading[4].split('.')
  index=int(sub_sub_heading[0])
  img=cv2.imread(i,0)
  img= Image.fromarray(img)
  img = img.resize((SIZE, SIZE))
  img_array.append(np.array(img))
  label.append(df[' hemorrhage'][index])

In [6]:
x=np.array(img_array)
x=x/255
y=np.array(label)

In [7]:
x=np.expand_dims(x,3)
x.shape

(200, 256, 256, 1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.15, random_state = 0)

In [9]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters =filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        
        # Define a conv2D_0, conv2D_1, etc based on the number of repetitions
        for i in range(self.repetitions):
            
            # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters,self.kernel_size,activation='relu',padding="same")
        
        # Define the max pool layer that will be added after the Conv2D blocks
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size, strides=strides)
  
    def call(self, inputs):
        # access the class's conv2D_0 layer
        conv2D_0 = vars(self)['conv2D_0']
        
        # Connect the conv2D_0 layer to inputs
        x = conv2D_0(inputs)

        # for the remaining conv2D_i layers from 1 to `repetitions` they will be connected to the previous layer
        for i in range(1,self.repetitions):
            # access conv2D_i by formatting the integer `i`. (hint: check how these were saved using `vars()` earlier)
            conv2D_i = vars(self)[f'conv2D_{i}']
            
            # Use the conv2D_i and connect it to the previous layer
            x = conv2D_i(x)

        # Finally, add the max_pool layer
        max_pool = self.max_pool(x)
        
        return max_pool

In [10]:
class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()

        # Creating blocks of VGG with the following 
        # (filters, kernel_size, repetitions) configurations
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)

        # Classification head
        # Define a Flatten layer
        self.flatten = tf.keras.layers.Flatten()
        # Create a Dense layer with 256 units and ReLU as the activation function
        self.fc = tf.keras.layers.Dense(256,activation='relu')
        # Finally add the softmax classifier using a Dense layer
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Chain all the layers one after the other
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

In [11]:
vgg = MyVGG(num_classes=2)
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vgg.fit(X_train,y_train,epochs=5)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.4964 - loss: 0.6992
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5793 - loss: 0.6882
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.6478 - loss: 0.6771
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5202 - loss: 0.6706
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5996 - loss: 0.6679


In [12]:
# Exceeds github's limit for file size so commented out for now
# vgg.save_weights("models/vgg.weights.h5")

In [13]:
y_pred=vgg.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [14]:
y_pred_binary=[]
for a in y_pred:
  y_pred_binary.append(np.argmax(a))

y_pred_binary=np.array(y_pred_binary)

In [15]:
from sklearn import metrics
print("Accuracy of model=",metrics.accuracy_score(y_test,y_pred_binary))

Accuracy of model= 0.7666666666666667


In [16]:
from sklearn.metrics import classification_report
target_names = ['0','1']
print(classification_report(y_test, y_pred_binary, target_names=target_names))

              precision    recall  f1-score   support

           0       0.75      0.80      0.77        15
           1       0.79      0.73      0.76        15

    accuracy                           0.77        30
   macro avg       0.77      0.77      0.77        30
weighted avg       0.77      0.77      0.77        30



In [17]:
pip install tf-keras-vis tensorflow


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install keras-vis


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
y_pred_list=y_pred_binary.tolist()

In [20]:
X_test_copy=X_test.copy()

In [21]:
from tf_keras_vis.utils.scores import CategoricalScore
score = CategoricalScore(y_pred_list)

In [22]:
## DOES NOT WORK
# from matplotlib import cm
# from tf_keras_vis.gradcam import Gradcam
# from tensorflow.keras.applications.vgg16 import preprocess_input
# import numpy as np

# X_test_rgb = np.repeat(X_test_copy, 3, axis=-1)

# input_images = preprocess_input(X_test_rgb)

# gradcam = Gradcam(vgg,
#                   model_modifier=None,
#                   clone=True)

# def score(output):
#     return output[:, 1]

# cam = gradcam(score,
#               input_images,
#               penultimate_layer=-1)


## Organized Code

In [23]:
import pandas as pd
import cv2
import tensorflow as tf 
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn import metrics
from sklearn.metrics import classification_report

In [24]:
def get_train_and_test_data():
  df=pd.read_csv("labels.csv")

  files=glob.glob("data/raw/hct/imgs/*.png")
  SIZE = 256

  label=[]
  img_array=[]
  for i in files:
    sub_heading=i.split('/')
    sub_sub_heading=sub_heading[4].split('.')
    index=int(sub_sub_heading[0])
    img=cv2.imread(i,0)
    img= Image.fromarray(img)
    img = img.resize((SIZE, SIZE))
    img_array.append(np.array(img))
    label.append(df[' hemorrhage'][index])

  x = np.array(img_array)
  x = x/255
  y = np.array(label)

  x = np.expand_dims(x,3)
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.15, random_state = 0)
  
  return X_train, X_test, y_train, y_test

In [25]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters =filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        
        # Define a conv2D_0, conv2D_1, etc based on the number of repetitions
        for i in range(self.repetitions):
            
            # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters,self.kernel_size,activation='relu',padding="same")
        
        # Define the max pool layer that will be added after the Conv2D blocks
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size, strides=strides)
  
    def call(self, inputs):
        # access the class's conv2D_0 layer
        conv2D_0 = vars(self)['conv2D_0']
        
        # Connect the conv2D_0 layer to inputs
        x = conv2D_0(inputs)

        # for the remaining conv2D_i layers from 1 to `repetitions` they will be connected to the previous layer
        for i in range(1,self.repetitions):
            # access conv2D_i by formatting the integer `i`. (hint: check how these were saved using `vars()` earlier)
            conv2D_i = vars(self)[f'conv2D_{i}']
            
            # Use the conv2D_i and connect it to the previous layer
            x = conv2D_i(x)

        # Finally, add the max_pool layer
        max_pool = self.max_pool(x)
        
        return max_pool

In [26]:
class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()

        # Creating blocks of VGG with the following 
        # (filters, kernel_size, repetitions) configurations
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)

        # Classification head
        # Define a Flatten layer
        self.flatten = tf.keras.layers.Flatten()
        # Create a Dense layer with 256 units and ReLU as the activation function
        self.fc = tf.keras.layers.Dense(256,activation='relu')
        # Finally add the softmax classifier using a Dense layer
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Chain all the layers one after the other
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

In [27]:
def get_model_accuracy(X_train, X_test, y_train, y_test):
  vgg = MyVGG(num_classes=2)
  vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  vgg.fit(X_train,y_train,epochs=5)

  # vgg.save_weights("models/vgg.weights.h5")

  y_pred=vgg.predict(X_test)

  y_pred_binary=[]
  for a in y_pred:
    y_pred_binary.append(np.argmax(a))

  y_pred_binary=np.array(y_pred_binary)

  accuracy = metrics.accuracy_score(y_test,y_pred_binary)

  print("Accuracy of model=",accuracy)

  print(classification_report(y_test, y_pred_binary, target_names=['0','1']))

  return accuracy

In [28]:
X_train, X_test, y_train, y_test = get_train_and_test_data()
accuracy = get_model_accuracy(X_train, X_test, y_train, y_test)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.4613 - loss: 0.6939
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.4985 - loss: 0.6873
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.6709 - loss: 0.6825
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5492 - loss: 0.6744
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.7069 - loss: 0.6689
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Accuracy of model= 0.5666666666666667
              precision    recall  f1-score   support

           0       1.00      0.13      0.24        15
           1       0.54      1.00      0.70        15

    accuracy                           0.57        30
   macro avg       0.77      0.57      0.47        30
weighted avg       0.77      0.57      0.47        30

